In [34]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec
import numpy as np  # Make sure that numpy is imported
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
model = Word2Vec.load("300features_40minwords_10context")

train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [35]:
type(model.wv.syn0)

numpy.ndarray

In [36]:
model.wv.syn0.shape

(16490, 300)

In [37]:
model["flower"]

array([  9.60964262e-02,   1.48660108e-01,  -7.05302460e-03,
        -3.80819328e-02,  -1.97653333e-03,   1.56424288e-02,
        -3.58977057e-02,   2.72922404e-02,   1.17391102e-01,
        -7.11113587e-02,   4.71835919e-02,   2.76274439e-02,
         5.82232065e-02,  -1.01643009e-03,   8.19908902e-02,
        -5.63554466e-02,  -7.95818307e-03,   2.60318792e-03,
         4.46205307e-03,   1.55150741e-02,  -1.01501904e-01,
         3.35950293e-02,   6.02126122e-02,   2.96408832e-02,
         1.44431870e-02,  -3.71579686e-03,   5.81015553e-03,
         5.89887910e-02,  -9.39505734e-03,   4.37740050e-02,
         3.63908224e-02,  -5.35665117e-02,  -2.96295024e-02,
         4.38000299e-02,   1.00670382e-03,  -4.54976037e-02,
         1.60219539e-02,  -4.19584215e-02,   1.32502364e-02,
         2.02988852e-02,  -7.62125850e-02,   3.87583040e-02,
         2.95232497e-02,  -3.98501530e-02,  -6.68181852e-02,
        -8.98217112e-02,  -3.51164155e-02,  -5.04063396e-03,
         3.02369185e-02,

In [47]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [48]:
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0.
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        if counter%1000. == 0.:
            print ("Review %d of %d" % (counter, len(reviews)))
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1.
    return reviewFeatureVecs

In [49]:
def review_to_wordlist( review, remove_stopwords=False ):
    review_text = BeautifulSoup(review,"lxml").get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

In [50]:
num_features = 300
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))

testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [51]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print ("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

result = forest.predict( testDataVecs )

output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [56]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = int(word_vectors.shape[0] / 5)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  739.755966424942 seconds.


In [58]:
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [66]:
for cluster in range(0,10):
    #
    # Print the cluster number  
    print ("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    value = list(word_centroid_map.values())
    key = list(word_centroid_map.keys())
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( value[i] == cluster ):
            words.append(key[i])
    print(words)


Cluster 0
['khan', 'salman', 'saif', 'aamir', 'shahrukh', 'kajol']

Cluster 1
['intervention', 'conversion', 'interference', 'delay', 'eliminating']

Cluster 2
['shields', 'brooke', 'selma']

Cluster 3
['employs', 'utilizing', 'expanding', 'employing', 'crafts']

Cluster 4
['water', 'cars', 'rain', 'snow', 'walls', 'horses', 'rocks', 'trees', 'buildings', 'rooms', 'objects', 'fog', 'sand', 'waves', 'stones', 'furniture', 'grass', 'clouds', 'skies', 'mist', 'floors', 'candles', 'overhead', 'sunlight', 'halls', 'forests']

Cluster 5
['scope']

Cluster 6
['pregnant', 'deceased', 'divorced', 'estranged', 'widowed', 'lila', 'grieving', 'distraught', 'orphaned', 'groom', 'aurora', 'delilah', 'heartbroken', 'prematurely', 'domineering', 'alexei']

Cluster 7
['screams', 'cries', 'yells', 'stabs', 'shouts', 'digs', 'screamed', 'barking', 'shouted', 'stared']

Cluster 8
['captain', 'commander', 'colonel', 'marine', 'vet', 'fugitive', 'patton', 'gen', 'roosevelt', 'col', 'marshal', 'admiral', 'g

In [67]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [68]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [70]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print ("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
